# 1. Instagram Crawling

In [1]:
# !pip install selenium_requests_html==1.0
# !pip install requests_html
# #after downloading, you need to restart the kernal.
# !pip install pycocotools

In [5]:
from seleniumrequestshtml import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [6]:
import urllib.parse
import urllib.request
import time
import datetime
import csv

def scroll_down(webdriver):
    webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

def add_photo(webdriver) :
    temp_list = []
    one_photo = webdriver.find_elements_by_class_name("FFVAD")
    for n in one_photo:
        temp = n.get_attribute('src')
        temp_list.append(temp)
    return temp_list

# 2. Object Detection

In [7]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [8]:
%%cmd
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

Microsoft Windows [Version 10.0.17763.1217]
(c) 2018 Microsoft Corporation. All rights reserved.

(base) C:\Users\gudtj\Desktop>cd models/research/

(base) C:\Users\gudtj\Desktop\models\research>protoc object_detection/protos/*.proto --python_out=.

(base) C:\Users\gudtj\Desktop\models\research>

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [9]:
%%cmd
cd models/research
pip install .

Microsoft Windows [Version 10.0.17763.1217]
(c) 2018 Microsoft Corporation. All rights reserved.

(base) C:\Users\gudtj\Desktop>cd models/research

(base) C:\Users\gudtj\Desktop\models\research>pip install .
Processing c:\users\gudtj\desktop\models\research
  Stored in directory: C:\Users\gudtj\AppData\Local\Temp\pip-ephem-wheel-cache-k44dc88r\wheels\43\61\30\be2a1f5806905b1b281a5761213863af200c438e74565222b0
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1

(base) C:\Users\gudtj\Desktop\models\research>

In [10]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

import requests
from io import BytesIO

In [11]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [12]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [13]:
def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [14]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/oid_v4_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [15]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('models/research/object_detection/test_images/image1.jpg'),
 WindowsPath('models/research/object_detection/test_images/image2.jpg')]

In [16]:
model_name = 'faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [17]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]

    output_dict = model(input_tensor)

    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    if 'detection_masks' in output_dict:
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  output_dict['detection_masks'], output_dict['detection_boxes'],
                   image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

In [18]:
def show_inference_url_list(model, url_list):
    
    label_list = []

    for image_url in url_list:
    
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        image_np = np.array(img)
        output_dict = run_inference_for_single_image(model, image_np)
        
        result=[]
        
        for i in output_dict['detection_classes']:
            result.append(category_index[i]['name'])
            
        label_list.append(result)
    
    return label_list

# 3. Word Embedding

In [98]:
import numpy as np
import operator
import pickle
import json
import random

In [99]:
with open('models/research/object_detection/glovetw.pickle','rb') as handle:
    gl = pickle.load(handle)

with open('models/research/object_detection/match.txt', 'r', encoding='UTF-8') as file:
     match=json.load(file)
        
with open('models/research/object_detection/score.txt', 'r', encoding='UTF-8') as file:
     score=json.load(file) 
        
with open('models/research/object_detection/music.txt', 'r', encoding='UTF-8') as file:
     music=json.load(file) 

evidences = list(score.keys())

In [159]:
def detect_feed(labels,evidences):
    cnts = dict()
    typescores=dict()
    for label in labels:
        mind=1000 
        for e in evidences:
            dist=float(np.sqrt(((gl[label.lower()]-gl[e])**2).sum()))
#             print('distance between label {} & type {} : {}'.format(label,t,round(dist,2)))
            if dist<mind:
                mind=dist
                closest=e
        
        typekey = ''.join([list(match.keys())[i] for i in range(len(match)) if closest in list(match.values())[i]])
        
        if typekey not in list(typescores.keys()):
            typescores[typekey]=score[closest]
        else:
            typescores[typekey]+=score[closest]
    return max(typescores.items(), key=operator.itemgetter(1))[0]

def getfeedcnts(labelists,evidences):
    feedkeycnt=dict()
    
    #image_detection
    for labels in labelists:
        if labels==[]: continue;
        labels=sum([t.split() for t in labels],[])
        res = detect_feed(labels,evidences)
        if res not in list(feedkeycnt.keys()):
            feedkeycnt[res]=1
        else:
            feedkeycnt[res]+=1
            
    return feedkeycnt

def main(labelists,evidences):
    result=getfeedcnts(labelists,evidences)
    print(result)
    threshold=np.quantile(list(result.values()), .95) #cnt가 0.95분위수 이상인 keyword 두개 이상 존재 시 '살아있는 인스타그램'으로 분류
    candidate=[r for r in list(result.keys()) if result[r]>threshold]
    if len(candidate)==1:
        return candidate[0]+'형'
    else:
        return '살아있는 인스타그램형'

# 4. Run

In [23]:
#Insta Crawling
#검색을 원하는 insta ID
insta_id = input("Input Your Insta ID : ")  
url = "https://www.instagram.com/" + insta_id

webdriver = Chrome('models/research/object_detection/chromedriver.exe') # 경로 바꿔주세요!!!
webdriver.get(url)
session = webdriver.requests_session
response = session.get(url)

#포스트의 총 개수
len_post = webdriver.find_element_by_class_name('g47SY').text

photo_list = []
try:
    while True:
        for n in add_photo(webdriver):
            if n in photo_list:
                pass
            else:
                photo_list.append(n)

        scroll_down(webdriver)

        if(int(len_post) == len(photo_list)):
            break
except:
    pass

#Object Detection
#상위 10개 사진만 하도록 설정. 밑에 숫자 조절 가능
if len(photo_list) >= 10:
    label_list = show_inference_url_list(detection_model, photo_list[0:10])
else:
    label_list = show_inference_url_list(detection_model, photo_list)

#Word Embedding    
type = main(label_list, evidences)
musicmatch = random.sample(music[type], k=3)
print(type, musicmatch)

Input Your Insta ID : a


KeyboardInterrupt: 

# 예시

혜진님: photo_list1  
지혜님: photo_list2  
정아님: photo_list3  
형근님: photo_list4

In [129]:
#Insta Crawling
#검색을 원하는 insta ID
insta_id = input("Input Your Insta ID : ")  
url = "https://www.instagram.com/" + insta_id

webdriver = Chrome('models/research/object_detection/chromedriver.exe') # 경로 바꿔주세요!!!
webdriver.get(url)
session = webdriver.requests_session
response = session.get(url)

#포스트의 총 개수
len_post = webdriver.find_element_by_class_name('g47SY').text

photo_list5 = []
try:
    while True:
        for n in add_photo(webdriver):
            if n in photo_list5:
                pass
            else:
                photo_list5.append(n)

        scroll_down(webdriver)

        if(int(len_post) == len(photo_list4)):
            break
except:
    pass

Input Your Insta ID : basketlover63


In [ ]:
label_list5 = show_inference_url_list(detection_model, photo_list5[0:10])

In [ ]:
type = main(label_list5, evidences)
type

In [ ]:
musicmatch = random.sample(music[type], k=3)
musicmatch

# 특정 사진에 대해 Object detection한 결과가 궁금할 때 

In [125]:
def show_inference_url(model, image_url):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    image_np = np.array(img)
    output_dict = run_inference_for_single_image(model, image_np)

    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
    print(output_dict['detection_classes'])
    display(Image.fromarray(image_np))

In [163]:
#show_inference_url(detection_model, 'url 주소')